## Group #3


1. Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [ ]:
# pip install rasterio
# %pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
# !pip install ipywidgets
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# pip install rasterstats
# %pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
# %pip install git+https://github.com/jgrss/geowombat  -q

In [ ]:

import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import mapping
import rasterio
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import os
from rasterio.mask import mask
from rasterstats import zonal_stats
import pandas as pd
#import geowombat as gw ## Chequear gw en collab



In [ ]:
import geopandas as gpd

# Ruta al shapefile
shapefile_path = r'../../_data/INEI_LIMITE_DEPARTAMENTAL'

# Leer el shapefile usando Geopandas
departmentos = gpd.read_file(shapefile_path)
departmentos = departments.to_crs('esri:54009')

print(departmentos.head())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Rutas en collab

raster = ["drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C12.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R11_C12.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C11.tif",
          "drive/content/diplomado/assignment_10/tif_files/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R12_C12.tif"
         ]

statistics = []

In [ ]:
# Iterar sobre cada archivo raster
for raster_path in raster:
    # estadísticas zonales
    stats = zonal_stats(departmentos, raster_path, stats="count sum", categorical=True, all_touched=True)
    # transforma DF
    stats_df = pd.DataFrame(stats)
    # Concatenar 
    df = pd.concat([departmentos.reset_index(drop=True), stats_df], axis=1)
    statistics.append(df)

In [ ]:
# Concatenar todos los DataFrames en all_stats en un único DataFrame
final_df = pd.concat(statistics, ignore_index=True)

print(final_df.head())

#Reemplazar todos los NaN por 0
final_df.fillna(0, inplace=True)

2. Then you are going to generate choropleth map using folium for these 15 variables.

In [ ]:
# Definir la área de pixeles (resolución del raster al cuadrado)
spatial_resolution = 10  # meters
pixel_area = spatial_resolution ** 2

# Iterar por cada una de las 15 categorías MSZ
for category in range(1, 16):
    # Crear una nueva columna para cada categoría MSZ
    column_name = f'MSZ_{category}_coverage'
    
    # Usar una función lambda con .apply para calcular el porcentaje de cobertura
    coverage_function = lambda row: ((row.get(str(category), 0) * pixel_area) / row['geometry'].area) * 100 if str(category) in row else 0
    
    # Aplicar función al DataFrame
    final_df[column_name] = final_df.apply(coverage_function, axis=1)

3. Save your html in the same folder of your JN. Name your HTML as your branch. This HTML should have all these layers. Please do not forget to use Layer Control.


In [ ]:
# Crear un mapa centrado en una ubicación específica (puedes ajustar esto según tus datos)
map_center = [-12.0757538, -76.9863174]
mymap = folium.Map(location=map_center, zoom_start=5)

# Agregar las capas al mapa
for category in range(1, 16):
    layer_name = f'MSZ_{category}_coverage'
    geojson_data = final_df.to_json()
    
    folium.Choropleth(
        geo_data=geojson_data,
        data=final_df,
        columns=['geometry', layer_name],
        key_on='feature.properties.geometry',
        fill_color='YlOrRd',  
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f'MSZ {category} Coverage'
    ).add_to(mymap)

# Agregar el control de capas al mapa
folium.LayerControl().add_to(mymap)

# Guardar el mapa como un archivo HTML
g3a10DFG_html = 'g3a10DFG.html'  
mymap.save(g3a10DFG_html)